In [1]:
#imports for data processing
import nltk
from nltk.stem.lancaster import LancasterStemmer
import numpy as np
import tflearn
import tensorflow as tf
import unicodedata
import sys
from tqdm import tqdm
import pandas as pd
import os
from random import shuffle
import string

#imports for nural network
from keras.models import Sequential
from keras.layers import *
from keras.optimizers import *
from keras.models import load_model

class SetData():
    def __init__(self):
        self.num_classes = 0
        self.classes = []
        self.train_data = None
        self.test_data = None
        self.train_data_dir = '/Users/arunprakash/Documents/TensorFlow/MovieData/train'
        self.test_data_dir = '/Users/arunprakash/Documents/TensorFlow/MovieData/test'
        for _, dirnames, _ in os.walk(self.train_data_dir):
            self.num_classes += len(dirnames)
            if len(dirnames)>0:
                self.classes = dirnames
        self.stemmer = LancasterStemmer()
        self.tbl = str.maketrans({key: None for key in string.punctuation})

    def one_hot_classes(self,clsname):
        ohe = []
        for i in self.classes:
            if i == clsname:
                ohe.append(1)
            else:
                ohe.append(0)
        return ohe

    def remove_punctuation(self,text):
        return text.translate(self.tbl)


    def train_data_with_label(self):
        train_text = []
        tr_words = []
        for cls in self.classes:
            train_data_path = self.train_data_dir+ '/' + cls
            for i in tqdm(os.listdir(train_data_path)):
                if i != '.DS_Store':
                    path = os.path.join(train_data_path, i)
                    df = pd.read_csv(path,delimiter='\n')
                    dfnp = df.values
                    for dlg in dfnp:
                        dlg = self.remove_punctuation(dlg[0])
                        dlg_tkn = nltk.word_tokenize(dlg)
                        tr_words.extend(dlg_tkn)
                        train_text.append((dlg_tkn, self.one_hot_classes(cls)))
        tr_words = [self.stemmer.stem(w.lower()) for w in tr_words]
        tr_words = sorted(list(set(tr_words)))
        shuffle(train_text)
        np.save('train_text.npy', train_text)
        np.save('train_text_wrds.npy', tr_words)
        return train_text, tr_words

    def test_data_with_label(self):
        test_text = []
        tst_words = []
        for cls in self.classes:
            test_data_path = self.test_data_dir+ '/' + cls
            for i in tqdm(os.listdir(test_data_path)):
                if i != '.DS_Store':
                    path = os.path.join(test_data_path, i)
                    df = pd.read_csv(path,delimiter='\n')
                    dfnp = df.values
                    for dlg in dfnp:
                        dlg = self.remove_punctuation(dlg[0])
                        dlg_tkn = nltk.word_tokenize(dlg)
                        tst_words.extend(dlg_tkn)
                        test_text.append((dlg_tkn, self.one_hot_classes(cls)))
        tst_words = [self.stemmer.stem(w.lower()) for w in tst_words]
        tst_words = sorted(list(set(tst_words)))
        np.save('test_text.npy', test_text)
        np.save('test_text_wrds.npy', tst_words)
        return test_text,tst_words

    def bag_of_words(self, data,wrds,data_type='train'):
        model_data = []
        batch_data = [] #for saving the bow as batches to overcome memory issue
        cnt=1
        for i in data:
            bow = []
            tokenized = i[0]            
            tokenized = [self.stemmer.stem(word.lower()) for word in tokenized]
            for w in wrds:
                bow.append(1) if w in tokenized else bow.append(0)
            model_data.append([bow, i[1]])
            batch_data.append([bow, i[1]])
            if cnt%5000 == 0:
                print('processing {}'.format(cnt))
                if cnt%50000 == 0:
                    batch_res = np.array(batch_data)
                    model_name = 'model_data_'+ data_type + str(cnt) +'_part.npy'
                    np.save(model_name, batch_res)
                    batch_data = []
            cnt += 1
        res = np.array(model_data)
        batch_res = np.array(batch_data)
        model_name = 'model_data_'+ data_type +'last_part.npy'
        np.save(model_name, batch_res)
        return res



/anaconda3/envs/tfdeeplearningv1.4/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)
/anaconda3/envs/tfdeeplearningv1.4/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
sd = SetData()

In [3]:
#Initialize data set
train_data, tr_words = sd.train_data_with_label()
test_data, tst_words = sd.test_data_with_label()

#Load data
#train_data = np.load('train_text.npy')
#tr_words = np.load('train_text_wrds.npy')
#test_data = np.load('test_text.npy')
#tst_words = np.load('test_text_wrds.npy')

100%|██████████| 6/6 [00:01<00:00,  5.49it/s]


In [5]:
training = sd.bag_of_words(train_data,tr_words,'train')
#training = np.load('model_data_50000train.npy')

training_data = list(training[:, 0])
training_label = list(training[:, 1])

processing 5000
processing 10000
processing 15000
processing 20000
processing 25000
processing 30000
processing 35000
processing 40000
processing 45000
processing 50000
processing 55000
processing 60000
processing 65000
processing 70000
processing 75000
processing 80000
processing 85000
processing 90000
processing 95000
processing 100000
processing 105000
processing 110000
processing 115000
processing 120000
processing 125000
processing 130000
processing 135000
processing 140000
processing 145000
processing 150000
processing 155000
processing 160000
processing 165000
processing 170000


In [7]:
testing = sd.bag_of_words(test_data,tr_words,'test')# here also we have to pass the training words
#testing = np.load('model_data_test.npy')

testing_data = list(testing[:, 0])
testing_label = list(testing[:, 1])

processing 5000
processing 10000
processing 15000
processing 20000
processing 25000
processing 30000
processing 35000
processing 40000
processing 45000


In [8]:
#tf.reset_default_graph()
#ANN
model = Sequential()

model.add(InputLayer(input_shape=[len(training_data[0])]))#keras will internally add batch dimention
model.add(Dense(128,activation='relu'))
model.add(Dense(128,activation='relu'))
model.add(Dense(64,activation='relu'))
model.add(Dense(32,activation='relu'))
model.add(Dropout(rate=0.5))
model.add(Dense(sd.num_classes,activation='softmax'))
optimizer = Adam(lr=1e-3)
model.compile(optimizer=optimizer,loss='categorical_crossentropy',metrics=['accuracy'])
model.fit(x=np.array(training_data),y=np.array(training_label),epochs=100,batch_size=200)
model.summary()

model.save('my_classify_model.h5')

Epoch 1/100
172630/172630 [==============================] - 100s 578us/step - loss: 1.5071 - acc: 0.3331
Epoch 2/100
172630/172630 [==============================] - 71s 413us/step - loss: 1.3571 - acc: 0.4310
Epoch 3/100
172630/172630 [==============================] - 71s 413us/step - loss: 1.2421 - acc: 0.4904
Epoch 4/100
172630/172630 [==============================] - 72s 418us/step - loss: 1.1305 - acc: 0.5433
Epoch 5/100
172630/172630 [==============================] - 72s 417us/step - loss: 1.0344 - acc: 0.5846
Epoch 6/100
172630/172630 [==============================] - 72s 416us/step - loss: 0.9617 - acc: 0.6135
Epoch 7/100
172630/172630 [==============================] - 71s 411us/step - loss: 0.9025 - acc: 0.6360
Epoch 8/100
172630/172630 [==============================] - 71s 414us/step - loss: 0.8586 - acc: 0.6550
Epoch 9/100
172630/172630 [==============================] - 71s 414us/step - loss: 0.8244 - acc: 0.6678
Epoch 10/100
172630/172630 [==========================

In [ ]:
loss_and_metrics = model.evaluate(np.array(testing_data), np.array(testing_label), batch_size=100)
print(loss_and_metrics)

In [ ]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
import numpy as np
import tflearn
import tensorflow as tf
import unicodedata
import sys
from tqdm import tqdm
import pandas as pd
import os
from random import shuffle
import string

#imports for nural network
from keras.models import Sequential
from keras.layers import *
from keras.optimizers import *
from keras.models import load_model

In [ ]:
model = load_model('my_classify_model.h5')

In [ ]:
sd = SetData()

In [ ]:
a =np.array(testing_data[1])

In [10]:
categories = sd.classes

In [ ]:
a.reshape(15685,)

In [75]:
print(categories[np.argmax(model.predict(np.array(testing_data[40000:40001])))])

drama


In [ ]:
model.predict(np.array(training_data[25000:25001]))

In [58]:
np.array(testing_data[40000:40001]).shape

(1, 15685)

In [25]:
testing_label[39999:40000]

[[0, 0, 0, 0, 1]]

In [ ]:
testing_data[40000:40001]

In [6]:
len(training_data)

172630

In [27]:
categories

['comedy', 'sciencefic', 'horror', 'action', 'drama']

In [44]:
def get_bow_for_new_sub(path):
    test_text = []
    df = pd.read_csv(path,delimiter='\n')
    dfnp = df.values
    for dlg in dfnp:
        dlg = sd.remove_punctuation(dlg[0])
        dlg_tkn = nltk.word_tokenize(dlg)
        test_text.append((dlg_tkn,[0]))
    return test_text

In [45]:
td = get_bow_for_new_sub('/Users/arunprakash/Documents/TensorFlow/MovieData/test/action/The.Rock.1996.720p.DVD9.BluRay.x264-SEPTiC_eng.txt')

In [46]:
td_bow = sd.bag_of_words(td,tr_words,data_type='my_test')

In [70]:
tdb = list(td_bow[:,0])

In [88]:
len(tdb)

2630

In [93]:
comedy = 0
sciencefic = 0
drama = 0
action = 0
horror = 0
for i in range(len(tdb)):
    pred = categories[np.argmax(model.predict(np.array(tdb[i:i+1])))]
    if pred == 'comedy':
        comedy += 1
    elif pred == 'sciencefic':
        sciencefic += 1
    elif pred == 'horror':
        horror += 1
    elif pred == 'action':
        action += 1
    elif pred == 'drama':
        drama += 1
print('comedy: '+ str(comedy) + ' sciencefic: '+ str(sciencefic) + ' horror: '+ str(horror) + ' action: '+ str(action) + ' drama: '+ str(drama))

comedy: 744 sciencefic: 458 horror: 359 action: 487 drama: 582


In [95]:
for cls in sd.classes:
    test_data_path = sd.test_data_dir+ '/' + cls
    print(cls+': ')
    for i in tqdm(os.listdir(test_data_path)):
        comedy = 0
        sciencefic = 0
        drama = 0
        action = 0
        horror = 0
        if i != '.DS_Store':
            #path = os.path.join(test_data_path, i)
            td = get_bow_for_new_sub(os.path.join(test_data_path, i))
            td_bow = sd.bag_of_words(td,tr_words,data_type='my_test')
            tdb = list(td_bow[:,0])
            for i in range(len(tdb)):
                pred = categories[np.argmax(model.predict(np.array(tdb[i:i+1])))]
                if pred == 'comedy':
                    comedy += 1
                elif pred == 'sciencefic':
                    sciencefic += 1
                elif pred == 'horror':
                    horror += 1
                elif pred == 'action':
                    action += 1
                elif pred == 'drama':
                    drama += 1
        print('comedy: '+ str(comedy) + ' sciencefic: '+ str(sciencefic) + ' horror: '+ str(horror) + ' action: '+ str(action) + ' drama: '+ str(drama))

  0%|          | 0/6 [00:00<?, ?it/s]

comedy: 


/anaconda3/envs/tfdeeplearningv1.4/lib/python3.5/site-packages/tqdm/_monitor.py:89: TqdmSynchronisationWarning: Set changed size during iteration (see https://github.com/tqdm/tqdm/issues/481)
  TqdmSynchronisationWarning)
 17%|█▋        | 1/6 [00:15<01:17, 15.59s/it]

comedy: 1074 sciencefic: 433 horror: 332 action: 336 drama: 387
comedy: 0 sciencefic: 0 horror: 0 action: 0 drama: 0


 50%|█████     | 3/6 [00:26<00:26,  8.96s/it]

...

comedy: 839 sciencefic: 373 horror: 447 action: 433 drama: 555


 83%|████████▎ | 5/6 [00:54<00:10, 10.92s/it]

comedy: 819 sciencefic: 340 horror: 354 action: 404 drama: 762


100%|██████████| 6/6 [01:10<00:00, 11.82s/it]

comedy: 691 sciencefic: 461 horror: 422 action: 417 drama: 764


In [1]:
import string

In [4]:
train_data

NameError: name 'train_data' is not defined